In [ ]:
# Hücre 1 (Başarılı Versiyonları Zorlayarak Kurulum)

# Lütfen bu hücreyi çalıştırmadan önce "Disconnect and delete runtime" ile
# Colab çalışma zamanını tamamen sıfırladığınızdan emin olun!

print("Başarılı olduğu bilinen kütüphane versiyonları kuruluyor...")

# 1. NumPy'yi 1.26.4'e sabitleyelim.
!pip install numpy==1.26.4 -q
print("NumPy 1.26.4 kuruldu.")

# 2. Datasets'i 3.6.0'a sabitleyelim.
!pip install datasets==3.6.0 -q
print("Datasets 3.6.0 kuruldu.")

# 3. Transformers'ı 4.48.3'e sabitleyelim.
!pip install transformers==4.48.3 -q
print("Transformers 4.48.3 kuruldu.")

# 4. Diğer yardımcı kütüphaneler
!pip install scikit-learn matplotlib seaborn -q
print("Scikit-learn, Matplotlib, Seaborn kuruldu/güncellendi.")

print("\nKütüphane kurulumları (Hücre 1) tamamlandı.")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
print("!!! LÜTFEN ŞİMDİ Colab Çalışma Zamanını (Runtime -> Restart runtime)      !!!")
print("!!! KESİNLİKLE YENİDEN BAŞLATIN.                                         !!!")
print("!!! Yeniden başlattıktan sonra Hücre 1'i TEKRAR ÇALIŞTIRMAYIN,           !!!")
print("!!! doğrudan Hücre 2'ye (Kütüphane Importları) geçin.                   !!!")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
# Hücre 2 (Düzeltilmiş): Kütüphanelerin Import Edilmesi, Versiyon Kontrolü ve Cihaz Belirleme

# Temel ve Hugging Face kütüphanelerinin import edilmesi
import transformers
import datasets
import torch
import numpy as np
import pandas as pd
import sklearn # scikit-learn'ün ana modülü
import matplotlib
import seaborn as sns
import fsspec # fsspec versiyonunu da kontrol edelim
import time   # Eğitim/çıkarım sürelerini ölçmek için
import sys    # Python versiyonunu almak için eklendi

# Hugging Face kütüphanelerinden sık kullanılacak modüller
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

# Cihazı belirleme: GPU varsa GPU, yoksa CPU kullanılacak.
# Colab'da GPU kullanmak için: Runtime -> Change runtime type -> Hardware accelerator -> GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# İsteğe bağlı: Daha temiz bir çıktı için uyarıları bastırma
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("Kütüphaneler başarıyla import edildi.")
print("-" * 50)
print("KULLANILAN KÜTÜPHANE VERSİYONLARI:")
# Python versiyonu sys modülü kullanılarak düzeltildi:
print(f"  Python Versiyonu (sys.version): {sys.version.split()[0]}")
print(f"  PyTorch: {torch.__version__}")
print(f"  Transformers: {transformers.__version__}")
print(f"  Datasets: {datasets.__version__}")
print(f"  Numpy: {np.__version__}")
print(f"  Pandas: {pd.__version__}")
print(f"  Scikit-learn: {sklearn.__version__}")
print(f"  Matplotlib: {matplotlib.__version__}")
print(f"  Seaborn: {sns.__version__}")
print(f"  fsspec: {fsspec.__version__}")
print("-" * 50)
print(f"KULLANILACAK CİHAZ: {device}")
print("-" * 50)

# GPU varsa, CUDA ve cuDNN versiyonlarını da yazdıralım (bilgi amaçlı)
if device.type == 'cuda':
    print(f"  CUDA Versiyonu (torch.version.cuda): {torch.version.cuda}")
    print(f"  cuDNN Versiyonu (torch.backends.cudnn.version()): {torch.backends.cudnn.version()}")
    print(f"  Kullanılabilir GPU Sayısı: {torch.cuda.device_count()}")
    print(f"  Aktif GPU Adı: {torch.cuda.get_device_name(0)}")
    print("-" * 50)
else:
    print("Uyarı: GPU bulunamadı veya aktif değil. Model eğitimi CPU üzerinde daha yavaş olacaktır.")
    print("Colab'da GPU'yu aktifleştirmek için 'Runtime' -> 'Change runtime type' menüsünü kullanabilirsiniz.")
    print("-" * 50)

In [ ]:
# Hücre 3: IMDB Veri Setinin Yüklenmesi ve Temel İnceleme

print("IMDB veri seti yükleniyor...")
try:
    imdb_dataset_raw = load_dataset("imdb")
    print("IMDB veri seti başarıyla yüklendi.")
    print("\nYüklenen Ham Veri Seti Yapısı:")
    print(imdb_dataset_raw)

    # label_feature'ı global yaparak diğer hücrelerden erişilebilir kılalım.
    # Bu, Hücre 2'de import edilen datasets kütüphanesini kullanır.
    global label_feature
    label_feature = imdb_dataset_raw['train'].features['label']

    print("\nEtiket Bilgisi (ClassLabel):")
    print(label_feature)
    print(f"Etiket 0: {label_feature.int2str(0)}") # neg
    print(f"Etiket 1: {label_feature.int2str(1)}") # pos

    # 'unsupervised' alt kümesini çıkararak yeni bir DatasetDict oluşturalım
    keys_to_keep = ['train', 'test']
    imdb_dataset_processed = datasets.DatasetDict(
        {k: imdb_dataset_raw[k] for k in keys_to_keep if k in imdb_dataset_raw}
    )

    if 'unsupervised' not in imdb_dataset_processed and 'unsupervised' in imdb_dataset_raw:
        print("\n'unsupervised' alt kümesi başarıyla işlenmiş veri setinden çıkarıldı.")
    elif 'unsupervised' not in imdb_dataset_raw:
        print("\n'unsupervised' alt kümesi zaten ham veri setinde bulunmuyordu.")
    else: # Bu durumun oluşmaması gerekir eğer pop veya yeniden oluşturma doğruysa
        print("\n'unsupervised' alt kümesi işlenmiş veri setinde hala mevcut veya bir sorun oluştu.")

    print("\nİşlenmiş (Relevant) Veri Seti Yapısı:")
    print(imdb_dataset_processed)

    if 'train' in imdb_dataset_processed and len(imdb_dataset_processed['train']) > 0:
        print("\nEğitim setinden ilk örnek:")
        example = imdb_dataset_processed['train'][0]
        print(f"  Metin: {example['text'][:200]}...") # İlk 200 karakter
        print(f"  Etiket: {example['label']} ({label_feature.int2str(example['label'])})")
    else:
        print("\nEğitim seti bulunamadı veya boş.")

except Exception as e:
    print(f"\nVeri seti yüklenirken veya işlenirken bir hata oluştu: {e}")
    # Hata durumunda değişkenlerin None olarak ayarlanması, sonraki hücrelerde kontrolü kolaylaştırır.
    imdb_dataset_processed = None
    if 'label_feature' in globals(): del label_feature # Eğer tanımlandıysa silelim

In [ ]:
# Hücre 4 (Düzeltilmiş - scikit-learn ile Bölme): Veri Setini Eğitim, Doğrulama ve Test Alt Kümelerine Ayırma

from sklearn.model_selection import train_test_split as sk_train_test_split

# Gerekli değişkenlerin bir önceki hücreden geldiğini varsayıyoruz
if 'imdb_dataset_processed' in globals() and imdb_dataset_processed is not None and 'train' in imdb_dataset_processed and \
   'label_feature' in globals() and label_feature is not None:
    print("Mevcut 'train' seti, scikit-learn kullanılarak 'validation' seti oluşturmak üzere bölünüyor...")

    train_texts_list = imdb_dataset_processed['train']['text']
    train_labels_list = imdb_dataset_processed['train']['label']

    new_train_texts, validation_texts, new_train_labels, validation_labels = sk_train_test_split(
        train_texts_list,
        train_labels_list,
        test_size=0.2,
        random_state=42,
        stratify=train_labels_list
    )

    original_features = imdb_dataset_processed['train'].features

    train_dataset_new = datasets.Dataset.from_dict(
        {'text': new_train_texts, 'label': new_train_labels},
        features=original_features
    )
    validation_dataset_new = datasets.Dataset.from_dict(
        {'text': validation_texts, 'label': validation_labels},
        features=original_features
    )

    # final_imdb_dataset'i global yapalım ki diğer hücrelerden erişilebilsin
    global final_imdb_dataset
    final_imdb_dataset = datasets.DatasetDict({
        'train': train_dataset_new,
        'validation': validation_dataset_new,
        'test': imdb_dataset_processed['test']
    })

    print("\nNihai Veri Seti Yapısı (Eğitim, Doğrulama, Test):")
    print(final_imdb_dataset)

    print(f"\nEğitim seti örnek sayısı: {len(final_imdb_dataset['train'])}")
    print(f"Doğrulama seti örnek sayısı: {len(final_imdb_dataset['validation'])}")
    print(f"Test seti örnek sayısı: {len(final_imdb_dataset['test'])}")

    def get_label_distribution(dataset_split, current_label_feature):
        if not dataset_split: return "Veri seti objesi None."
        if len(dataset_split) == 0: return "Veri seti boş."
        try:
            labels = dataset_split['label']
            series = pd.Series(labels).value_counts().sort_index()
            # current_label_feature'ın None olup olmadığını ve int2str metoduna sahip olup olmadığını kontrol et
            if current_label_feature is not None and hasattr(current_label_feature, 'int2str'):
                 return series.rename(index=current_label_feature.int2str)
            return series
        except Exception as e: return f"Etiket dağılımı hesaplanırken hata: {e}"

    print("\nEğitim Seti Etiket Dağılımı:")
    print(get_label_distribution(final_imdb_dataset['train'], label_feature))

    print("\nDoğrulama Seti Etiket Dağılımı:")
    print(get_label_distribution(final_imdb_dataset['validation'], label_feature))

    print("\nTest Seti Etiket Dağılımı:")
    print(get_label_distribution(final_imdb_dataset['test'], label_feature))
else:
    print("Hata: 'imdb_dataset_processed' veya 'label_feature' doğru şekilde tanımlanmamış. Bu hücre çalıştırılamadı.")
    if 'final_imdb_dataset' in globals(): del final_imdb_dataset

In [ ]:
# Hücre 5: BERT Tokenizer'ının Yüklenmesi ve Veri Setinin Tokenizasyonu

from transformers import AutoTokenizer

model_checkpoint = "bert-base-uncased"
print(f"Kullanılacak model checkpoint: {model_checkpoint}")

# final_imdb_dataset'in Hücre 4'te tanımlandığını varsayıyoruz
if 'final_imdb_dataset' in globals() and final_imdb_dataset is not None:
    try:
        # tokenizer'ı global yapalım ki diğer hücrelerden erişilebilsin
        global tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
        print(f"\n'{model_checkpoint}' için tokenizer başarıyla yüklendi.")

        sample_text = "Hello, this is a sample sentence for tokenization!"
        encoded_sample = tokenizer(sample_text)
        print(f"\nÖrnek metin: '{sample_text}'")
        print(f"Tokenize edilmiş hali (input_ids): {encoded_sample['input_ids']}")
        print(f"Token ID'lerinin tekrar metne dönüştürülmüş hali: {tokenizer.convert_ids_to_tokens(encoded_sample['input_ids'])}")

        def tokenize_function(examples):
            return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

        print(f"\nVeri seti tokenize ediliyor (max_length=256)... Bu işlem biraz zaman alabilir.")

        # tokenized_datasets'i global yapalım
        global tokenized_datasets
        tokenized_datasets = final_imdb_dataset.map(
            tokenize_function,
            batched=True,
            remove_columns=["text"]
        )

        tokenized_datasets.set_format("torch")

        print("\nVeri seti başarıyla tokenize edildi ve formatı PyTorch tensörlerine ayarlandı.")
        print("Tokenize edilmiş veri seti yapısı:")
        print(tokenized_datasets)

        if 'train' in tokenized_datasets and len(tokenized_datasets['train']) > 0:
            print("\nTokenize edilmiş eğitim setinden ilk örnek:")
            example_item = tokenized_datasets['train'][0]
            print(example_item)
            if 'input_ids' in example_item and hasattr(example_item['input_ids'], '__len__'):
                 print(f"Input IDs uzunluğu: {len(example_item['input_ids'])}")
            else:
                print("Örnekte 'input_ids' bulunamadı veya uzunluğu alınamadı.")

    except Exception as e:
        print(f"\nTokenizer yüklenirken veya tokenizasyon sırasında bir genel hata oluştu: {e}")
        if 'tokenized_datasets' in globals(): del tokenized_datasets
        if 'tokenizer' in globals(): del tokenizer
else:
    print("Hata: 'final_imdb_dataset' bulunamadığı için tokenizasyon işlemi başlatılamadı.")
    if 'tokenized_datasets' in globals(): del tokenized_datasets
    if 'tokenizer' in globals(): del tokenizer

In [ ]:
# Hücre 6: Önceden Eğitilmiş BERT Modelinin Yüklenmesi

from transformers import AutoModelForSequenceClassification

# model_checkpoint, num_labels, device değişkenlerinin önceki hücrelerde tanımlandığını varsayıyoruz
# num_labels'ı burada tekrar tanımlayalım veya global olduğundan emin olalım.
# Hücre 5'te model_checkpoint, Hücre 2'de device tanımlandı.
# num_labels'ı burada tanımlamak daha güvenli olabilir.
num_labels = 2

if 'model_checkpoint' in globals() and 'num_labels' in globals() and 'device' in globals():
    print(f"'{model_checkpoint}' modeli '{num_labels}' etiket ile sınıflandırma görevi için yükleniyor...")
    try:
        # model'i global yapalım
        global model
        model = AutoModelForSequenceClassification.from_pretrained(
            model_checkpoint,
            num_labels=num_labels
        )

        model.to(device)

        print(f"\nModel başarıyla yüklendi ve '{device}' cihazına taşındı.")

        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"\nModeldeki toplam parametre sayısı: {total_params:,}")
        print(f"Modeldeki eğitilebilir parametre sayısı: {trainable_params:,}")

        if hasattr(model.config, 'num_labels'):
            print(f"Modelin yapılandırmasındaki etiket sayısı: {model.config.num_labels}")
        if hasattr(model, 'classifier') and hasattr(model.classifier, 'out_features'):
             print(f"Sınıflandırıcı katmanının çıktı boyutu: {model.classifier.out_features}")

    except Exception as e:
        print(f"\nModel yüklenirken bir hata oluştu: {e}")
        if 'model' in globals(): del model
else:
    print("Hata: Gerekli değişkenler ('model_checkpoint', 'num_labels', 'device') bulunamadı.")
    if 'model' in globals(): del model

In [ ]:
# Hücre 7 (Epoch Bazında Değerlendirme ile Güncellenmiş TrainingArguments - Düzeltilmiş `if` koşulu ile):
# Eğitim Argümanlarının Tanımlanması, Metrik Hesaplama Fonksiyonu ve Modelin Eğitilmesi

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
# numpy, torch, time zaten Hücre 2'de import edilmişti.
# num_labels, model, tokenized_datasets, tokenizer, device değişkenlerinin
# önceki hücrelerde doğru şekilde tanımlandığını ve erişilebilir olduğunu varsayıyoruz.

# 1. Performans Metriklerini Hesaplama Fonksiyonu
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # num_labels'ın (Hücre 6'da tanımlanmıştı) bu scope'ta erişilebilir olması gerekir.
    # Ya global yapın ya da model.config.num_labels kullanın.
    # Şimdilik modelin config'inden alalım, daha güvenli.
    current_num_labels = model.config.num_labels if 'model' in globals() and model else 2 # Fallback

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)

    try:
        cm = confusion_matrix(labels, preds, labels=[0, 1])
        tn, fp, fn, tp = cm.ravel()
    except ValueError:
        tn, fp, fn, tp = 0,0,0,0
        if len(np.unique(labels)) == 1:
            if np.unique(labels)[0] == 0 and (len(np.unique(preds))==0 or (len(np.unique(preds))==1 and np.unique(preds)[0] == 0)): tn = len(labels)
            if np.unique(labels)[0] == 1 and (len(np.unique(preds))==0 or (len(np.unique(preds))==1 and np.unique(preds)[0] == 1)): tp = len(labels)

    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    auc_value = 0.0
    if pred.predictions.ndim == 2 and pred.predictions.shape[1] == current_num_labels:
        try:
            logits_tensor = torch.tensor(pred.predictions)
            probs = torch.softmax(logits_tensor, dim=-1).cpu().numpy()
            positive_class_probs = probs[:, 1]
            auc_value = roc_auc_score(labels, positive_class_probs)
        except ValueError:
            auc_value = 0.0

    return {
        'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1,
        'specificity': specificity, 'auc': auc_value
    }

# 2. Eğitim Argümanları (TrainingArguments) - Epoch bazında değerlendirme ile
# Önceki çalışmanızdaki output_dir'den farklı bir isim kullanalım ki sonuçlar karışmasın.
training_args_epoch_eval = TrainingArguments(
    output_dir="./results_epoch_evaluation", # Yeni çıktı dizini
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",             # <<-- HER EPOCH SONUNDA DEĞERLENDİRME
    save_strategy="epoch",                   # <<-- HER EPOCH SONUNDA CHECKPOINT KAYDETME
    load_best_model_at_end=True,             # <<-- Eğitim sonunda en iyi modeli yükle
    metric_for_best_model="accuracy",        # En iyi modeli 'accuracy' metriğine göre belirle
    save_total_limit=2, # En iyi ve son checkpoint'i tutabilir
    report_to="tensorboard",
)

print(f"\nEğitim argümanları (epoch bazında değerlendirme ile) tanımlandı. Çıktı dizini: {training_args_epoch_eval.output_dir}")

# 3. Trainer Objesinin Oluşturulması - DÜZELTİLMİŞ `if` KOŞULU
if 'model' in globals() and model is not None and \
   'tokenized_datasets' in globals() and tokenized_datasets is not None and \
   'tokenizer' in globals() and tokenizer is not None: # 'device' ve 'training_time' kontrolü çıkarıldı

    # trainer'ı global yapalım ki sonraki hücrelerde (örn: Hücre 8) kullanılabilsin
    global trainer
    trainer = Trainer(
        model=model,
        args=training_args_epoch_eval,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )
    print("\nTrainer objesi başarıyla oluşturuldu.")

    print("\nModel eğitimi (epoch bazında değerlendirme ile) başlatılıyor...")
    # device ve diğer bilgilerin Hücre 2'de zaten yazdırıldığını varsayıyoruz.
    print(f"Epoch sayısı: {training_args_epoch_eval.num_train_epochs}, Train Batch Size: {training_args_epoch_eval.per_device_train_batch_size}")

    # training_time'ı global yapalım
    global training_time
    start_time_train_epoch_eval = time.time()
    try:
        train_result = trainer.train()
        end_time_train_epoch_eval = time.time()
        training_time = end_time_train_epoch_eval - start_time_train_epoch_eval

        print(f"\nEğitim tamamlandı! Toplam eğitim süresi: {training_time:.2f} saniye ({training_time/60:.2f} dakika).")

        if hasattr(train_result, 'metrics') and train_result.metrics:
            print("Genel eğitim sonuç metrikleri (trainer.train() dönüşünden):")
            for key, value in train_result.metrics.items():
                print(f"  {key}: {value}")

        # load_best_model_at_end=True olduğu için, trainer.model artık en iyi modeli içeriyor.
        # Bu en iyi modeli kaydedelim.
        best_model_path = f"{training_args_epoch_eval.output_dir}/best_model"
        trainer.save_model(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"Eğitim sonrası (en iyi) model ve tokenizer '{best_model_path}' adresine kaydedildi.")

    except Exception as e:
        print(f"\nModel eğitimi sırasında bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
        if 'train_result' not in globals(): train_result = None
        training_time = None # Hata durumunda training_time'ı None yapalım
else:
    print("\nModel, tokenize edilmiş veri seti veya tokenizer bulunamadığı için Trainer oluşturulamadı ve eğitim başlatılamadı.")
    if 'trainer' in globals(): del trainer
    if 'train_result' not in globals(): train_result = None
    training_time = None

In [ ]:
# Hücre 8: Eğitilmiş Modelin Test Seti Üzerinde Değerlendirilmesi

print("Eğitilmiş model test seti üzerinde değerlendiriliyor...")

# Gerekli değişkenlerin varlığını kontrol edelim
if 'trainer' in globals() and trainer is not None and \
   'tokenized_datasets' in globals() and "test" in tokenized_datasets:
    try:
        # eval_dataset parametresi ile test setini belirtiyoruz
        test_metrics_output = trainer.evaluate(eval_dataset=tokenized_datasets["test"])

        print("\nTest Seti Performans Metrikleri:")
        # trainer.evaluate() metriklerin başına 'eval_' ekler
        print(f"  Test Kaybı (Loss): {test_metrics_output.get('eval_loss', 'N/A'):.4f}")
        print(f"  Test Accuracy: {test_metrics_output.get('eval_accuracy', 'N/A'):.4f}")
        print(f"  Test Precision: {test_metrics_output.get('eval_precision', 'N/A'):.4f}")
        print(f"  Test Recall: {test_metrics_output.get('eval_recall', 'N/A'):.4f}")
        print(f"  Test F1-Score: {test_metrics_output.get('eval_f1', 'N/A'):.4f}")
        print(f"  Test Specificity: {test_metrics_output.get('eval_specificity', 'N/A'):.4f}")
        print(f"  Test AUC: {test_metrics_output.get('eval_auc', 'N/A'):.4f}")

        # Metrikleri globalde saklayalım ki Hücre 9'da kullanılabilsin
        global bert_test_metrics
        bert_test_metrics = test_metrics_output

    except Exception as e:
        print(f"\nTest seti değerlendirmesi sırasında bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
        bert_test_metrics = None # Hata durumunda None olarak ayarla
else:
    print("Hata: 'trainer' objesi veya 'tokenized_datasets['test']' bulunamadı.")
    print("Lütfen önceki hücrelerin doğru çalıştığından emin olun.")
    bert_test_metrics = None

In [ ]:
# Hücre 9: Karmaşıklık Matrisi ve ROC Eğrisinin Çizdirilmesi (Test Seti Üzerinden)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.metrics import auc as sklearn_auc # sklearn.metrics.auc'yi import ediyoruz
# numpy ve torch zaten import edilmişti

# Gerekli değişkenlerin varlığını ve doğruluğunu kontrol edelim
if 'trainer' in globals() and trainer is not None and \
   'tokenized_datasets' in globals() and "test" in tokenized_datasets and \
   'bert_test_metrics' in globals() and bert_test_metrics is not None and \
   'label_feature' in globals() and label_feature is not None and hasattr(label_feature, 'names') and hasattr(label_feature, 'int2str'):

    print("Test seti üzerinde tahminler alınıyor...")
    try:
        test_predictions_output = trainer.predict(tokenized_datasets["test"])

        logits = test_predictions_output.predictions
        probabilities = torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()
        predicted_labels = np.argmax(logits, axis=1)
        true_labels = test_predictions_output.label_ids

        # 1. Karmaşıklık Matrisi (Confusion Matrix)
        # Etiketlerin [0, 1] olduğunu ve label_feature.names'in ['neg', 'pos'] olduğunu varsayıyoruz
        cm_labels_numeric = [0, 1] # Sayısal etiketler
        cm_display_labels = label_feature.names # Görüntülenecek etiketler

        cm = confusion_matrix(true_labels, predicted_labels, labels=cm_labels_numeric)

        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=cm_display_labels,
                    yticklabels=cm_display_labels)
        plt.title('Karmaşıklık Matrisi (Test Seti)')
        plt.xlabel('Tahmin Edilen Etiket')
        plt.ylabel('Gerçek Etiket')
        plt.show()

        print("\nKarmaşıklık Matrisi Değerleri:")
        # cm[0,0]=TN (neg-neg), cm[0,1]=FP (neg-pos), cm[1,0]=FN (pos-neg), cm[1,1]=TP (pos-pos)
        print(f"  Doğru Negatif (TN) [{cm_display_labels[0]}-{cm_display_labels[0]}]: {cm[0,0]}")
        print(f"  Yanlış Pozitif (FP) [{cm_display_labels[0]}-{cm_display_labels[1]}]: {cm[0,1]}")
        print(f"  Yanlış Negatif (FN) [{cm_display_labels[1]}-{cm_display_labels[0]}]: {cm[1,0]}")
        print(f"  Doğru Pozitif (TP) [{cm_display_labels[1]}-{cm_display_labels[1]}]: {cm[1,1]}")

        # 2. ROC Eğrisi
        # Pozitif sınıfın ID'sini alalım (genellikle 1)
        positive_class_id = label_feature.str2int('pos') if hasattr(label_feature, 'str2int') else 1

        # Sadece pozitif sınıfın olasılıklarını alalım
        positive_class_probabilities = probabilities[:, positive_class_id]

        fpr, tpr, thresholds = roc_curve(true_labels, positive_class_probabilities, pos_label=positive_class_id)
        roc_auc_value_sklearn = sklearn_auc(fpr, tpr)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC eğrisi (AUC = {roc_auc_value_sklearn:.4f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') # Rastgele tahmin çizgisi
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Yanlış Pozitif Oranı (1 - Specificity)')
        plt.ylabel('Doğru Pozitif Oranı (Recall/Sensitivity)')
        plt.title('Alıcı İşletim Karakteristiği (ROC) Eğrisi')
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.show()

        print(f"\nHücre 8'de hesaplanan Test AUC (trainer.evaluate): {bert_test_metrics.get('eval_auc', 'N/A'):.4f}")
        print(f"Bu hücrede ROC eğrisi için hesaplanan AUC (sklearn): {roc_auc_value_sklearn:.4f}")

    except Exception as e:
        print(f"\nTahminler alınırken veya görselleştirmeler oluşturulurken bir hata oluştu: {e}")
        import traceback
        print(traceback.format_exc())
else:
    print("Hata: Gerekli değişkenler ('trainer', 'tokenized_datasets['test']', 'bert_test_metrics', 'label_feature') bulunamadı veya 'label_feature' doğru yapılandırılmamış.")
    print("Lütfen önceki hücrelerin doğru çalıştığından emin olun.")

In [ ]:
# Hücre 10 (Geliştirilmiş Log İşleme ve Tam Veriyle Grafikler):
# Eğitim ve Doğrulama Kayıp/Metrik Grafikleri

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# trainer ve log_history'nin Hücre 7'den geldiğini varsayıyoruz
if 'trainer' in globals() and trainer is not None and \
   hasattr(trainer.state, 'log_history') and trainer.state.log_history:

    log_history = trainer.state.log_history

    print("Ham log_history içeriği (ilk 3 ve son 3 kayıt - eğer yeterliyse):")
    # log_history'nin yapısını daha iyi anlamak için birkaç giriş yazdıralım
    num_entries_to_show = 3
    if len(log_history) > 2 * num_entries_to_show:
        for i in range(num_entries_to_show): print(log_history[i])
        print("...")
        for i in range(len(log_history) - num_entries_to_show, len(log_history)): print(log_history[i])
    else:
        for entry in log_history: print(entry)
    print("-" * 50)

    # log_history'yi bir DataFrame'e dönüştürelim
    log_df = pd.DataFrame(log_history)

    # Eğitim logları: 'loss' anahtarını içerir ve 'eval_loss' anahtarını içermez
    # Ayrıca, 'learning_rate' içerenler genellikle eğitim adımı loglarıdır.
    train_logs_df = log_df[log_df['loss'].notna() & log_df['eval_loss'].isna()].copy()

    # Değerlendirme logları: 'eval_loss' anahtarını içerir
    eval_logs_df = log_df[log_df['eval_loss'].notna()].copy()

    # Grafikleri çizdirelim
    if not train_logs_df.empty and not eval_logs_df.empty:
        plt.figure(figsize=(12, 6))

        # Eğitim Kaybı
        plt.plot(train_logs_df['epoch'], train_logs_df['loss'], 'b-o', label='Eğitim Kaybı (Training Loss)')

        # Doğrulama Kaybı
        plt.plot(eval_logs_df['epoch'], eval_logs_df['eval_loss'], 'r-s', label='Doğrulama Kaybı (Validation Loss)')

        plt.title('Epoch Bazında Eğitim ve Doğrulama Kaybı')
        plt.xlabel('Epoch')
        plt.ylabel('Kayıp (Loss)')
        plt.legend()
        plt.grid(True)

        # X ekseni için epoch değerlerini belirle
        # Epochlar genellikle tam sayılar (1.0, 2.0, 3.0 ...) veya bunlara çok yakın ondalıklar olur.
        # En son log_history'nizde epochlar tam sayıydı.
        epochs_present = sorted(list(set(train_logs_df['epoch'].round().tolist() + eval_logs_df['epoch'].round().tolist())))
        if epochs_present:
            min_e, max_e = min(epochs_present), max(epochs_present)
            plt.xticks(np.arange(int(min_e), int(max_e) + 1, 1.0))
            plt.xlim(left=int(min_e) - 0.5, right=int(max_e) + 0.5) # Grafiğin kenarlarında biraz boşluk bırak

        plt.show()

        # Doğrulama doğruluğu grafiği
        if 'eval_accuracy' in eval_logs_df.columns and not eval_logs_df['eval_accuracy'].isna().all():
            plt.figure(figsize=(12, 6))
            plt.plot(eval_logs_df['epoch'], eval_logs_df['eval_accuracy'], 'g-^', label='Doğrulama Doğruluğu (Validation Accuracy)')
            plt.title('Epoch Bazında Doğrulama Doğruluğu')
            plt.xlabel('Epoch')
            plt.ylabel('Doğruluk (Accuracy)')
            plt.legend()
            plt.grid(True)
            if epochs_present: # Aynı x-ekseni ayarını kullanalım
                plt.xticks(np.arange(int(min_e), int(max_e) + 1, 1.0))
                plt.xlim(left=int(min_e) - 0.5, right=int(max_e) + 0.5)
            plt.show()
        else:
            print("\nDoğrulama doğruluğu ('eval_accuracy') loglarda anlamlı şekilde bulunamadı veya tüm değerler NaN.")
            print("Mevcut doğrulama logları (epoch ve eval_accuracy sütunları):")
            if 'epoch' in eval_logs_df and 'eval_accuracy' in eval_logs_df:
                print(eval_logs_df[['epoch', 'eval_accuracy']].to_string())
            elif 'epoch' in eval_logs_df:
                 print(eval_logs_df[['epoch']].to_string())
            else:
                print(eval_logs_df.to_string())

    elif train_logs_df.empty:
        print("Eğitim logları ('loss' içeren ve 'eval_loss' içermeyen) filtrelenemedi. Ham log_df:")
        print(log_df.to_string())
    elif eval_logs_df.empty:
        print("Doğrulama logları ('eval_loss' içeren) filtrelenemedi. Ham log_df:")
        print(log_df.to_string())
    else:
        print("Grafik çizdirmek için yeterli eğitim veya doğrulama logu bulunamadı.")

else:
    print("Hata: 'trainer' objesi veya 'log_history' bulunamadı.")
    print("Lütfen önceki hücrelerin (özellikle Hücre 7 - eğitim) doğru çalıştığından emin olun.")

In [ ]:
# Hücre 11: Eğitim ve Çıkarım Sürelerinin Hesaplanması

# time, torch, tqdm.auto, DataLoader zaten import edilmiş olmalı
from tqdm.auto import tqdm
from torch.utils.data import DataLoader

# 1. Eğitim Süresi
calculated_training_time_value = None
if 'training_time' in globals() and training_time is not None: # Hücre 7'de global yapılmıştı
    calculated_training_time_value = training_time
elif 'trainer' in globals() and trainer is not None and \
     hasattr(trainer.state, 'log_history') and trainer.state.log_history:
    for log_entry in reversed(trainer.state.log_history):
        if 'train_runtime' in log_entry: # Trainer'ın logladığı genel eğitim süresi
            calculated_training_time_value = log_entry['train_runtime']
            break

if calculated_training_time_value is not None:
    print(f"Toplam Model Eğitim Süresi: {calculated_training_time_value:.2f} saniye ({calculated_training_time_value/60:.2f} dakika)")
else:
    print("Eğitim süresi bilgisi `training_time` değişkeninden veya `log_history`'den alınamadı.")

# 2. Örnek Başına Ortalama Çıkarım Süresi (Test Seti Üzerinden)
avg_inference_time_per_sample = None
# Gerekli değişkenlerin varlığını kontrol edelim
if 'model' in globals() and model is not None and \
   'tokenized_datasets' in globals() and "test" in tokenized_datasets and \
   'device' in globals() and 'tokenizer' in globals() and tokenizer is not None :

    model.eval() # Modeli değerlendirme moduna al (dropout vb. katmanları etkisizleştirir)

    # Modelin beklediği giriş sütunlarını belirle
    if hasattr(tokenizer, 'model_input_names'):
        model_input_names = tokenizer.model_input_names
    else:
        model_input_names = ['input_ids', 'attention_mask', 'token_type_ids']
        if 'token_type_ids' not in tokenized_datasets["test"].features: # Eğer veri setinde yoksa listeden çıkar
            if 'token_type_ids' in model_input_names: model_input_names.remove('token_type_ids')

    try:
        # Sadece modelin ihtiyaç duyduğu sütunları içeren bir test seti kopyası oluştur
        # ve formatını torch olarak ayarla
        inference_ready_dataset = tokenized_datasets["test"].select_columns(model_input_names)
        inference_ready_dataset.set_format("torch")
    except Exception as e_cols: # select_columns yoksa veya başka bir hata olursa
        print(f"Uyarı: Çıkarım için sütun seçimi/formatlamada sorun ({e_cols}). Eski yöntem denenecek.")
        try:
            current_test_set_columns = tokenized_datasets["test"].column_names
            cols_to_remove_for_inference = [col for col in current_test_set_columns if col not in model_input_names]
            inference_ready_dataset = tokenized_datasets["test"].remove_columns(cols_to_remove_for_inference)
            inference_ready_dataset.set_format("torch")
        except Exception as e_format_fallback:
            print(f"Formatlama için fallback de başarısız: {e_format_fallback}. Orijinal test seti kullanılacak.")
            inference_ready_dataset = tokenized_datasets["test"] # Son çare


    inference_batch_size = 32 # Eğitimdeki eval_batch_size ile aynı olabilir
    inference_dataloader = DataLoader(inference_ready_dataset, batch_size=inference_batch_size)

    total_inference_time_val = 0.0 # İsim çakışmasını önlemek için farklı bir isim
    num_samples_processed = 0

    print(f"\nTest seti üzerinde çıkarım süresi hesaplanıyor (Batch Size: {inference_batch_size})...")

    with torch.no_grad(): # Gradyan hesaplamalarını kapat
        for batch in tqdm(inference_dataloader, desc="Çıkarım Yapılıyor"):
            inputs_for_model = {}
            valid_batch = True
            for name in model_input_names:
                if name in batch:
                    inputs_for_model[name] = batch[name].to(device)
                else: # Beklenen girdi batch'te yoksa
                    # print(f"Uyarı: Beklenen girdi '{name}' batch'te bulunamadı.")
                    # Bu durum genellikle collate_fn veya dataset formatıyla ilgilidir.
                    # Eğer input_ids yoksa, bu batch'i atlamak daha güvenli olabilir.
                    if name == 'input_ids': valid_batch = False
                    break

            if not valid_batch or not inputs_for_model :
                try: num_in_batch = len(batch.get('input_ids', [])) # input_ids varsa onunla say
                except: num_in_batch = 0
                num_samples_processed += num_in_batch # Atlananları da sayıma ekleyelim (ya da işlemeyelim)
                                                    # Şimdilik, eğer input_ids yoksa bu batch'i sayıma eklemeyelim.
                if num_in_batch > 0 and not valid_batch : print(f"Uyarı: Geçerli model girdisi eksik, {num_in_batch} örnek içeren bu yığın atlandı.")
                continue

            current_batch_size = len(inputs_for_model['input_ids'])
            num_samples_processed += current_batch_size

            start_batch_time = time.perf_counter()
            outputs = model(**inputs_for_model)
            end_batch_time = time.perf_counter()

            batch_time = end_batch_time - start_batch_time
            total_inference_time_val += batch_time

    if num_samples_processed > 0 and total_inference_time_val > 0 :
        avg_inference_time_per_sample = total_inference_time_val / num_samples_processed
        samples_per_second_inference = num_samples_processed / total_inference_time_val
        print(f"\nToplam {num_samples_processed} örnek için çıkarım süresi: {total_inference_time_val:.4f} saniye")
        print(f"Örnek başına ortalama çıkarım süresi: {avg_inference_time_per_sample:.6f} saniye/örnek")
        print(f"Saniyede işlenen örnek sayısı (çıkarım): {samples_per_second_inference:.2f} örnek/saniye")
    elif num_samples_processed > 0 and total_inference_time_val == 0 :
        print(f"\nToplam {num_samples_processed} örnek için çıkarım süresi çok kısa (< ölçülebilir hassasiyet), hız çok yüksek.")
        avg_inference_time_per_sample = 0.0
    else:
        print("Çıkarım için hiç örnek işlenemedi veya süre ölçülemedi.")

else:
    print("Hata: Model, tokenize edilmiş test seti, tokenizer veya cihaz bilgisi bulunamadı.")